## Домашнее задание №9
### Генерация поэзии с помощью нейронных сетей: шаг 1
##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

Ваша основная задача: научиться генерироват стихи с помощью простой рекуррентной нейронной сети (Vanilla RNN). В качестве корпуса текстов для обучения будет выступать роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина.

In [1]:
# do not change the code in the block below
# __________start of block__________
import string
import os
from random import sample

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F

from IPython.display import clear_output

import matplotlib.pyplot as plt
%matplotlib inline
# __________end of block__________

In [2]:
# do not change the code in the block below
# __________start of block__________
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("{} device is available".format(device))
# __________end of block__________

cuda device is available


#### 1. Загрузка данных.

In [6]:
# do not change the code in the block below
# __________start of block__________
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt

with open('onegin.txt', 'r', encoding='utf-8') as iofile:
    text = iofile.readlines()

text = "".join([x.replace('\t\t', '').lower() for x in text])
# __________end of block__________

"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


#### 2. Построение словаря и предобработка текста
В данном задании требуется построить языковую модель на уровне символов. Приведем весь текст к нижнему регистру и построим словарь из всех символов в доступном корпусе текстов. Также добавим токен `<sos>`.

In [8]:
# do not change the code in the block below
# __________start of block__________
tokens = sorted(set(text.lower())) + ["<sos>"]
num_tokens = len(tokens)

assert num_tokens == 84, "Check the tokenization process"

token_to_idx = {x: idx for idx, x in enumerate(tokens)}
idx_to_token = {idx: x for idx, x in enumerate(tokens)}

assert len(tokens) == len(token_to_idx), "Mapping should be unique"

print("Seems fine!")


text_encoded = [token_to_idx[x] for x in text]
# __________end of block__________

Seems fine!


__Ваша задача__: обучить классическую рекуррентную нейронную сеть (Vanilla RNN) предсказывать следующий символ на полученном корпусе текстов и сгенерировать последовательность длины 100 для фиксированной начальной фразы.

Вы можете воспользоваться кодом с занятие №6 или же обратиться к следующим ссылкам:
* Замечательная статья за авторством Andrej Karpathy об использовании RNN: [link](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* Пример char-rnn от Andrej Karpathy: [github repo](https://github.com/karpathy/char-rnn)
* Замечательный пример генерации поэзии Шекспира: [github repo](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb)

Данное задание является достаточно творческим. Не страшно, если поначалу оно вызывает затруднения. Последняя ссылка в списке выше может быть особенно полезна в данном случае.

Далее для вашего удобства реализована функция, которая генерирует случайный батч размера `batch_size` из строк длиной `seq_length`. Вы можете использовать его при обучении модели.

In [9]:
# do not change the code in the block below
# __________start of block__________
batch_size = 256
seq_length = 100
start_column = np.zeros((batch_size, 1), dtype=int) + token_to_idx["<sos>"]


def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size * seq_length - 1)
    data = np.array(
        text_encoded[start_index : start_index + batch_size * seq_length]
    ).reshape((batch_size, -1))
    yield np.hstack((start_column, data))


# __________end of block__________

Пример батча:

In [11]:
next(generate_chunk())

array([[83, 59, 57, ...,  0, 64,  1],
       [83, 56, 45, ...,  7,  0, 47],
       [83, 62, 50, ..., 51, 49, 45],
       ...,
       [83, 47, 59, ..., 55, 45, 60],
       [83, 53, 63, ..., 26, 26,  0],
       [83,  0, 55, ...,  1, 58, 50]])

Далее вам предстоит написать код для обучения модели и генерации текста.

In [12]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = CharRNN(vocab_size=num_tokens, hidden_size=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
loss_fn = nn.CrossEntropyLoss()

n_epochs = 20

for epoch in range(n_epochs):
    model.train()
    hidden = model.init_hidden(batch_size).to(device)

    for batch in generate_chunk():
        x = torch.tensor(batch[:, :-1], dtype=torch.long).to(device)
        y = torch.tensor(batch[:, 1:], dtype=torch.long).to(device)

        optimizer.zero_grad()
        output, hidden = model(x, hidden.detach())
        loss = loss_fn(output.reshape(-1, num_tokens), y.reshape(-1))
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 4.4532
Epoch 2, Loss: 4.1200
Epoch 3, Loss: 3.7261
Epoch 4, Loss: 3.3555
Epoch 5, Loss: 3.1758
Epoch 6, Loss: 3.0721
Epoch 7, Loss: 3.0363
Epoch 8, Loss: 2.9615
Epoch 9, Loss: 2.9054
Epoch 10, Loss: 2.8633
Epoch 11, Loss: 2.8399
Epoch 12, Loss: 2.7836
Epoch 13, Loss: 2.8007
Epoch 14, Loss: 2.7586
Epoch 15, Loss: 2.7203
Epoch 16, Loss: 2.6928
Epoch 17, Loss: 2.6647
Epoch 18, Loss: 2.6932
Epoch 19, Loss: 2.6331
Epoch 20, Loss: 2.6124


Шаблон функции `generate_sample` также доступен ниже. Вы можете как дозаполнить его, так и написать свою собственную функцию с нуля. Не забывайте, что все примеры в обучающей выборке начинались с токена `<sos>`.

In [ ]:
def generate_sample(
    char_rnn, seed_phrase=None, max_length=500, temperature=1.0, device=device
):
    """
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs,
                        smaller temperature converges to the single most likely output
    """
    if seed_phrase is not None:
        x_sequence = [token_to_idx["<sos>"]] + [
            token_to_idx[token] for token in seed_phrase
        ]
    else:
        x_sequence = [token_to_idx["<sos>"]]

    x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)

    char_rnn.eval()
    hidden = char_rnn.init_hidden(1).to(device)
    
    for i in range(x_sequence.shape[1] - 1):
        _, hidden = char_rnn(x_sequence[:, i:i+1], hidden)

    input_char = x_sequence[:, -1:]

    while x_sequence.shape[1] < max_length:
        out, hidden = char_rnn(input_char, hidden)
        logits = out[:, -1, :] / temperature
        probs = torch.softmax(logits, dim=-1)
        next_char = torch.multinomial(probs, 1)
        x_sequence = torch.cat([x_sequence, next_char], dim=1)
        input_char = next_char

    return "".join([tokens[ix] for ix in x_sequence.cpu().data.numpy()[0]])

Пример текста сгенерированного обученной моделью доступен ниже. Не страшно, что в тексте много несуществующих слов. Используемая модель очень проста: это простая классическая RNN.

In [39]:
print(
    generate_sample(
        model, " мой дядя самых честных правил", max_length=500, temperature=0.8
    )
)

<sos> мой дядя самых честных правила
жова,
москомед
ранесц опой пененный ваихтаянитвавсожнод дреец, потьной каз ой гой илевеет падан чене
ковь,
мог глапыем пут ватокой босрака прем едиума тет, сти гровувюнойи
о стредгой
ков стале  свеголопил, бовоце понь:
о пождавы вальен межсь свелим дузннечный пхогукой но поготь п м ол,



у вогоскени мний стогомиковлонпожиль мрестой вове я отьви строви постовестугол вжиньт нои жтал празог транне вунир
 медо подны,
ивкони м проз теобывех тевеко на нат мед, псыре у


### Сдача задания
Сгенерируйте десять последовательностей длиной 500, используя строку ' мой дядя самых честных правил'. Температуру для генерации выберите самостоятельно на основании визуального качества генериуремого текста. Не забудьте удалить все технические токены в случае их наличия.

Сгенерированную последовательность сохрание в переменную `generated_phrase` и сдайте сгенерированный ниже файл в контест.

In [40]:
seed_phrase = " мой дядя самых честных правил"

In [ ]:
generated_phrases = [
    generate_sample(
        model,
        seed_phrase=seed_phrase,
        max_length=501,
        temperature=0.8
    ).replace('<sos>', '')
    for _ in range(10)
]

In [45]:
# do not change the code in the block below
# __________start of block__________

if "generated_phrases" not in locals():
    raise ValueError("Please, save generated phrases to `generated_phrases` variable")

for phrase in generated_phrases:

    if not isinstance(phrase, str):
        raise ValueError("The generated phrase should be a string")

    if len(phrase) != 500:
        raise ValueError("The `generated_phrase` length should be equal to 500")

    assert all(
        [x in set(tokens) for x in set(list(phrase))]
    ), "Unknown tokens detected, check your submission!"


submission_dict = {"token_to_idx": token_to_idx, "generated_phrases": generated_phrases}

np.save("submission_dict_hw09.npy", submission_dict, allow_pickle=True)
print("File saved to `submission_dict_hw09.npy`")
# __________end of block__________

File saved to `submission_dict_hw09.npy`


На этом задание завершено. Поздравляем!